<a href="https://colab.research.google.com/github/nikxlvii/pytorch/blob/main/breast_cancer_diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
import torch
from torch.autograd import Variable

In [3]:
data = load_breast_cancer()
X = data['data']
y = data['target']

In [4]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.3,random_state = 24)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val,test_size=0.5,random_state = 24)

In [5]:
X_train = Variable(torch.from_numpy(X_train))
X_val = Variable(torch.from_numpy(X_val))
y_train = Variable(torch.from_numpy(y_train))
y_val = Variable(torch.from_numpy(y_val))
X_test = Variable(torch.from_numpy(X_test))
y_test = Variable(torch.from_numpy(y_test))

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(30,20)
    self.fc2 = nn.Linear(20,10)
    self.fc3 = nn.Linear(10,5)
    self.fc4 = nn.Linear(5,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.sigmoid(self.fc4(x))

    return x

In [7]:
net = Net()

In [8]:
import torch.optim as optim

criterion = nn.BCELoss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [9]:
# training loop

num_epochs = 300

for epoch in range(num_epochs):
  running_loss = 0.0
  y_pred = net(X_train.float())
  loss = criterion(y_pred, y_train.view(-1, 1).float())

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  running_loss += loss.item()


print('Finished Training')

Finished Training


In [10]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

y_pred_val = net(X_val.float())
y_pred_val_binary = (y_pred_val >= 0.5).float()

precision, recall, f1, _ = precision_recall_fscore_support(y_val.view(-1, 1).numpy(), y_pred_val_binary.detach().numpy(), average='binary')

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')

print('\nClassification Report:')
print(classification_report(y_val.view(-1, 1).numpy(), y_pred_val_binary.detach().numpy()))


Precision: 0.942
Recall: 0.942
F1 Score: 0.942

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.91      0.91        33
           1       0.94      0.94      0.94        52

    accuracy                           0.93        85
   macro avg       0.93      0.93      0.93        85
weighted avg       0.93      0.93      0.93        85



In [11]:
pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=98e96bc53c2fc3c900fd9bb45a7ea18d6a44bebb74bcf42cd9da8411cac145e2
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [13]:
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(X_train.numpy(),feature_names=data.feature_names,class_names=data.target_names,discretize_continuous = True)

In [14]:
# we will take two test cases, one benign and the other malignant

benign_idx = np.where(y_test.numpy() == 1)[0][0]
malignant_idx = np.where(y_test.numpy() == 0)[0][0]